In [ ]:
import pandas as pd

# Preprocessing

In [ ]:
#import train and test csvs
train_csv = pd.read_csv("drive/MyDrive/garanti/train_users.csv")
test_csv = pd.read_csv("drive/MyDrive/garanti/test_users.csv")

### Skills csv
num of skills for every person

In [ ]:
#import skills cs
skills_csv = pd.read_csv("drive/MyDrive/garanti/skills.csv")

In [ ]:
# make sum of skills df
skills_sum = skills_csv.groupby(['user_id'], sort=False).size().reset_index(name='num_of_skills')
skills_sum.head()

,user_id,num_of_skills
0,1,2
1,2,6
2,3,3
3,5,11
4,6,6


In [ ]:
# add column on train
train_csv = pd.merge(train_csv, skills_sum, on='user_id')
train_csv.head()

,user_id,industry,location,moved_after_2019,num_of_skills
0,1301,Information Technology and Services,"Istanbul, Istanbul, Turkey",1,23
1,6950,Internet,"Istanbul, Istanbul, Turkey",0,49
2,4880,Online Media,Turkey,0,21
3,26046,Telecommunications,"Istanbul, Istanbul, Turkey",0,30
4,11005,Banking,"Istanbul, Turkey",0,48


In [ ]:
# add column on test
test_csv = pd.merge(test_csv, skills_sum, on='user_id')
test_csv.head()

,user_id,industry,is_from_turkey,num_of_school,english,other,turkish,num_of_skills
0,33967,7,1,4,1,2,1,31
1,55082,3,1,2,1,0,1,37
2,37165,1,1,2,1,0,1,4
3,42841,1,1,1,1,1,1,25
4,17053,7,1,4,1,0,1,47


### Work_experience
num of experience

In [ ]:
#import
work_experiences_csv = pd.read_csv('drive/MyDrive/garanti/work_experiences.csv')

In [ ]:
#drop other columns
work_drop = work_experiences_csv.drop(['location','start_year_month'],axis=1)
work_drop.head()

,user_id,company_id
0,53442,2651
1,34558,815
2,63761,26354
3,10738,89
4,8711,3113


In [ ]:
#create a df for num of experience 
experience_sum = work_experiences_csv.groupby(['user_id'], sort=False).size().reset_index(name='num_of_experience')
experience_sum.head()

,user_id,num_of_experience
0,53442,4
1,34558,6
2,63761,3
3,10738,3
4,8711,5


In [ ]:
# add column on train
train_csv = pd.merge(train_csv, experience_sum, on='user_id')
train_csv.head()

,user_id,industry,location,moved_after_2019,num_of_skills,num_of_experience
0,1301,Information Technology and Services,"Istanbul, Istanbul, Turkey",1,23,4
1,6950,Internet,"Istanbul, Istanbul, Turkey",0,49,6
2,4880,Online Media,Turkey,0,21,4
3,26046,Telecommunications,"Istanbul, Istanbul, Turkey",0,30,4
4,11005,Banking,"Istanbul, Turkey",0,48,3


In [ ]:
# add column on test
test_csv = pd.merge(test_csv, experience_sum, on='user_id')
test_csv.head()

,user_id,industry,is_from_turkey,num_of_school,english,other,turkish,num_of_skills,num_of_experience
0,33967,7,1,4,1,2,1,31,2
1,55082,3,1,2,1,0,1,37,2
2,37165,1,1,2,1,0,1,4,1
3,42841,1,1,1,1,1,1,25,2
4,17053,7,1,4,1,0,1,47,6


### Train csv
make 1 if location equals Turkey, 0 for others. Fill nulls with mode value on from turkey column.

#### location column

In [ ]:
def get_country(location_str, target_country):
    if location_str is None or pd.isnull(location_str):
          return None
    countries = location_str.split(',')
    if target_country in countries:
        return 1
    else:
        return 0

In [ ]:
# remove spaces like between city an countries
train_csv['location'] = train_csv['location'].str.replace(' ', '')

In [ ]:
train_csv['is_from_turkey'] = train_csv['location'].apply(get_country, target_country='Turkey')
train_csv.head()

,user_id,industry,location,moved_after_2019,num_of_skills,num_of_experience,is_from_turkey
0,1301,Information Technology and Services,"Istanbul,Istanbul,Turkey",1,23,4,1.0
1,6950,Internet,"Istanbul,Istanbul,Turkey",0,49,6,1.0
2,4880,Online Media,Turkey,0,21,4,1.0
3,26046,Telecommunications,"Istanbul,Istanbul,Turkey",0,30,4,1.0
4,11005,Banking,"Istanbul,Turkey",0,48,3,1.0


In [ ]:
#check if we have nulls on location
null_location_ids = train_csv.loc[train_csv['is_from_turkey'].isnull(), 'user_id']
null_location_ids

34945    65204
Name: user_id, dtype: int64

In [ ]:
#check value counts for find mode
counts = train_csv['is_from_turkey'].value_counts()
print(counts)

1.0    44145
0.0      126
Name: is_from_turkey, dtype: int64


In [ ]:
#fill nulls with mode
train_csv['is_from_turkey'] = train_csv['is_from_turkey'].fillna(1)  # Fill null values with the mode value

In [ ]:
#it should be 0
train_csv.is_from_turkey.isnull().sum()

0

In [ ]:
#drop location column
train_csv = train_csv.drop('location', axis=1)
train_csv.head()

,user_id,industry,moved_after_2019,num_of_skills,num_of_experience,is_from_turkey
0,1301,Information Technology and Services,1,23,4,1.0
1,6950,Internet,0,49,6,1.0
2,4880,Online Media,0,21,4,1.0
3,26046,Telecommunications,0,30,4,1.0
4,11005,Banking,0,48,3,1.0


#### industry column

In [ ]:
unique_values = train_csv['industry'].value_counts()

# Print number of unique values
print("Number of unique values:", len(unique_values))

# Print unique values and their counts
print("Unique values and their counts:")
print(unique_values)

Number of unique values: 133
Unique values and their counts:
Computer Software                      8426
Information Technology and Services    5912
Telecommunications                     1456
Defense & Space                        1266
Internet                                929
                                       ... 
Wine and Spirits                          1
Printing                                  1
Package/Freight Delivery                  1
Animation                                 1
Facilities Services                       1
Name: industry, Length: 133, dtype: int64


In [ ]:
#find most working industries
count_max_industries = unique_values[unique_values >= 800]

# Print the industries with a count of 1
print(count_max_industries.index.tolist())

['Computer Software', 'Information Technology and Services', 'Telecommunications', 'Defense & Space', 'Internet', 'Banking']


In [ ]:
#make min industries as 1
train_csv.loc[~train_csv['industry'].isin(['Banking', 'Internet', 'Defense & Space', 'Telecommunications', 'Information Technology and Services', 'Computer Software']), 'industry'] = 1

In [ ]:
#change most common industries
train_csv['industry'] = train_csv['industry'].replace({'Banking': 2, 'Internet': 3, 'Defense & Space': 4, 'Telecommunications':5, 'Information Technology and Services':6, 'Computer Software':7})

### Test csv
create is_from_turket column for test csv


In [ ]:
test_csv = pd.read_csv("drive/MyDrive/garanti/test_users.csv")

#### location column

In [ ]:
# remove spaces like between city an countries
test_csv['location'] = test_csv['location'].str.replace(' ', '')

In [ ]:
test_csv['is_from_turkey'] = test_csv['location'].apply(get_country, target_country='Turkey')
test_csv.head()

,user_id,industry,location,is_from_turkey
0,17449,Research,Turkey,1
1,33967,Computer Software,"Istanbul,Istanbul,Turkey",1
2,2110,Automotive,Turkey,1
3,55082,Internet,Turkey,1
4,37165,Electrical/Electronic Manufacturing,Turkey,1


In [ ]:
#check if we have nulls on is_from_turkey
null_location_ids = test_csv.loc[test_csv['is_from_turkey'].isnull(), 'user_id']
null_location_ids

Series([], Name: user_id, dtype: int64)

dont have nulls on that column so I dont need to fill anything.

In [ ]:
#drop location column
test_csv = test_csv.drop('location', axis=1)

#### industry column

In [ ]:
test_csv.loc[~test_csv['industry'].isin(['Banking', 'Internet', 'Defense & Space', 'Telecommunications', 'Information Technology and Services', 'Computer Software']), 'industry'] = 1

In [ ]:
test_csv['industry'] = test_csv['industry'].replace({'Banking': 2, 'Internet': 3, 'Defense & Space': 4, 'Telecommunications':5, 'Information Technology and Services':6, 'Computer Software':7})

In [ ]:
test_csv = pd.merge(test_csv, edu_sum, on='user_id')

In [ ]:
test_csv = pd.merge(test_csv, result, on='user_id')

In [ ]:
test_csv.head()

,user_id,industry,is_from_turkey,num_of_school,english,other,turkish
0,33967,7,1,4,1,2,1
1,55082,3,1,2,1,0,1
2,37165,1,1,2,1,0,1
3,42841,1,1,1,1,1,1
4,17053,7,1,4,1,0,1


### Education.csv
* num of school

#### if degree not in turkey

In [ ]:
education_csv = pd.read_csv("drive/MyDrive/garanti/education.csv")
education_csv.head()

,user_id,school_name,degree,fields_of_study,start_year_month,end_year_month
0,0,Anadolu Üniversitesi,NaN,NaN,NaN,NaN
1,0,Adıyaman Üniversitesi,NaN,NaN,NaN,NaN
2,0,Fırat Üniversitesi,NaN,NaN,NaN,NaN
3,1,Fırat Üniversitesi,Yüksek Lisans,Yazılım Mühendisliği,NaN,NaN
4,1,Fırat Üniversitesi,Lisans,Yazılım Mühendisliği,NaN,NaN


In [ ]:
education_csv['school_name'] = education_csv['school_name'].str.lower()
school_names = education_csv.school_name.unique()
school_names

array(['anadolu üniversitesi', 'adıyaman üniversitesi',
       'fırat üniversitesi', ..., 'sezin primary school',
       'the george washington university - school of business',
       "saint mary's university"], dtype=object)

In [ ]:
school_names = pd.DataFrame(school_names, columns=['name'])

In [ ]:
# Drop rows if the 'school' column contains either 'lisesi' or 'üniversite'
mask = school_names['name'].str.contains('lisesi|üniversite|okul|ç|ü|ı|ş|ğ|ö|teknik|meslek|anadolu|kolej|firat|konya|proje|istanbul|tekno|science high school|primary school|adnan|--')
school_names = school_names.drop(school_names[mask].index)

In [ ]:
# Print the values in the 'school' column that contain 'High School'
print(school_names[school_names['name'].str.contains('.')]['name'])

5        the state university of applied sciences in el...
8                              rigas tehniska universitate
19                                   university of houston
24                         lublin university of technology
32                      bialystok university of technology
                               ...                        
10681                      saint joseph french high school
10682             sainte pulchérie french secondary school
10683                    ca'​ foscari university of venice
10687    the george washington university - school of b...
10688                              saint mary's university
Name: name, Length: 4146, dtype: object


In [ ]:
school_names.to_excel('school.xlsx', index=False)

#### num of schools

In [ ]:
selected_columns = ['user_id', 'school_name']
edu = education_csv[selected_columns]
edu_sum = edu.groupby(['user_id'], sort=False).size().reset_index(name='num_of_school')
edu_sum.head()

,user_id,num_of_school
0,0,3
1,1,2
2,2,2
3,3,1
4,4,1


In [ ]:
train_csv = pd.merge(train_csv, edu_sum, on='user_id')
test_csv = pd.merge(train_csv, edu_sum, on='user_id')
train_csv.head()

,user_id,industry,moved_after_2019,num_of_skills,num_of_experience,is_from_turkey,english,other,turkish,num_of_school
0,1301,Information Technology and Services,1,23,4,1.0,1,0,1,2
1,6950,Internet,0,49,6,1.0,1,2,0,1
2,26046,Telecommunications,0,30,4,1.0,1,0,0,3
3,11005,Banking,0,48,3,1.0,1,0,1,2
4,13722,Mobile Games,1,12,1,1.0,1,0,0,3


### Language

In [ ]:
languages_csv = pd.read_csv("drive/MyDrive/garanti/languages.csv")

In [ ]:
languages_csv.head()

,user_id,language,proficiency
0,8,İngilizce,full_professional
1,8,Türkçe,native_or_bilingual
2,8,Fransızca,elementary
3,10,ingilizce,NaN
4,11,Turkish,native_or_bilingual


In [ ]:
#create new dataframe to fix it
# Get the set of user_ids in train and test DataFrames
train_user_ids = set(train_csv['user_id'])
test_user_ids = set(test_csv['user_id'])

# Create a boolean mask for rows in education that have user_id in train or test
mask = languages_csv['user_id'].isin(train_user_ids) | languages_csv['user_id'].isin(test_user_ids)

# Use the mask to index the education DataFrame, keeping only the rows with user_id in train or test
languages = languages_csv[mask]

In [ ]:
languages.head()

,user_id,language,proficiency
3,10,ingilizce,NaN
4,11,Turkish,native_or_bilingual
5,11,English,professional_working
6,11,Kurdish,native_or_bilingual
7,12,Türkçe,native_or_bilingual


In [ ]:
languages.proficiency.unique()

array([nan, 'native_or_bilingual', 'professional_working',
       'full_professional', 'limited_working', 'elementary'], dtype=object)

In [ ]:
#make this column numeric
languages['proficiency'] = languages['proficiency'].replace({'native_or_bilingual':3 ,'professional_working':3,
                                                             'full_professional':3, 'limited_working':2, 'elementary':1})
languages.head()

<ipython-input-89-2e1e5fdd6132>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  languages['proficiency'] = languages['proficiency'].replace({'native_or_bilingual':3 ,'professional_working':3,


,user_id,language,proficiency
3,10,ingilizce,NaN
4,11,Turkish,3.0
5,11,English,3.0
6,11,Kurdish,3.0
7,12,Türkçe,3.0


In [ ]:
# make lower case
languages['language'] = languages['language'].str.lower()

<ipython-input-90-2020a6660ed2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  languages['language'] = languages['language'].str.lower()


In [ ]:
english = ['ingilizce', 'english', 
       'i̇ngilizce',
       'i̇ngilizce, orta (1100-1500)',
       'ingilzce', 'i̇ngilizce ', 'i̇ngizce',
       'i̇ngilizce, eski (yaklaşık 450-1100)', 'i̇ngilizce, upper intermediate',
       'i̇ngi̇li̇zce', 'orta düzey i̇ngilizce',
       'english, middle (1100-1500)',
       'english (upper-intermediate)',
       'english-b2 upper intermediate', 'i̇ngilizce %30',
       'i̇ngilizce, i̇yi(1100-1500)', 'english(advanced)',
       'english ',
       'ingilizce - başlangıç', 'i̇nglizce', 'ingilizce ', 'english, advanced', 'english (us)',
       'i̇ngilizce, i̇yi',
       'englisch',
       'english (advanced)',
       'english - global village sydney australia', 'advanced english', 'i̇ngilizce,','2- english',
       'i̇nglilizce',
       'i̇ngilizce, orta/i̇leri düzey',
       'i̇ngilizce (c1)',
       'i̇ngilizce, i̇yi (1100-1500)', 'inglizce',
       'i̇ngilzce',
       'i̇ngilizce, i̇leri (2500-3000)',
       'i̇ngilizce, b1', 'i̇ngilizce (b1 - b2)', 'i̇ngi̇li̇zce ',
       'i̇ngilice',
       'i̇ngilizce(orta)', 'engilish,', 'i̇ngilize',
       'english, old (ca.450-1100)',
       'i̇ngilizce/english',
       'english (b2, upper-intermediate)',
       'englis',
       'ingilizce(english)', 'i̇ngilizce(b2)', 'i̇ngizice',
       'i̇ngilizce, ileri', 'en',
       'i̇ngilizce (upper-intermediate)',
       'i̇ngilizce (english)', 'ielts',
       'english - (yds : 93,75)', 'i̇ngiliz',
       'i̇ngilizce - ielts 7',
       'i̇ngilizce(orta seviye)', 'i̇ngilizce pre-intermediate',
       'i̇ngilizce ( orta düzeyde )',
       'english, deutsch', 'mesleki i̇ngilizce', 'i̇ngilizce (orta)',
       'english (intermediate)',
       'i̇ngilizce iyi','ingles',
       'i̇ngilizce / english','i̇ngilizce (advanced)',
       'i̇ngilizce, i̇yi düzeyde',
       'i̇ngilizce-intermediate', 'mesleki i̇ng.','english,', 'i̇ngilizce(c1)',
       'i̇ngilizice',
       'i̇ngilizce, i̇yi ', 'ingilizce (b2)',
       'i̇ngilizce ( toeic - 725 )', 'i̇ngilizce a2',
        '■ english ■',
       'i̇ng',
       'i̇ngilizce, b2',
       'i̇ngilizce- bels english school. as upper intermediate',
       'i̇ngilizce, i̇yi (1500-2000)', 'i̇ngilizce (excellent)',
       'ing', 'engish',
       'english uk', 'english us', 'english, pre-advance', '-english',
       'english c1', 'english upper intermediate']

In [ ]:
languages['language'] = languages['language'].replace(english, 'english')

<ipython-input-95-5ab6dac0c1c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  languages['language'] = languages['language'].replace(english, 'english')


array(['english', 'turkish', 'kurdish', 'türkçe', 'almanca', 'japanese',
       'i̇spanyolca', 'greek',
       'almanca, orta yüksek (yaklaşık 1050-1500)', 'türkmence', 'korece',
       'rusça', 'romanca', 'lehçe', 'polish', 'japonca', 'arabic',
       'azerice', 'german', 'arapça', 'francais', 'italiano', 'farsça',
       'russian', 'kürtçe', 'arabish', 'spanish', 'portuguese',
       'germanic languages', 'french', 'chinese', 'bulgarian', 'español',
       'ermenice', 'fransızca', 'turkish-mother language', 'français',
       'bulgarca', 'armenian', 'german(beginner)', 'romanian',
       'azerbaijani', 'ukrainian', 'i̇talyanca',
       'turkish, ottoman (1500-1928)', 'çekçe ', 'boşnakça',
       'almanca, eski yüksek (yaklaşık 750-1050)', 'çince', 'italian',
       'türkçe, osmanlıca (1500-1928)', 'türkish', 'özbekçe', 'turkmence',
       'lehçe polish', 'python', 'deutsch', 'rusca', 'kazakça', 'persian',
       'ottoman turkish', 'korean', 'esperanto', '학국의', 'farsca', 'c#',
       

In [ ]:
#replace spaces wiht nothing
languages.language = languages.language.replace(' ','')

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
turkish = ['turkish', 'türkçe', 'turkish-mother language', 'türkish',
        'turkce', 'turkçe',
       'türkisch', 'turkish (native)',
       '3- turkish',
       'turkısh', 'türkçe,',
       'turc',
       'turkish ', 'türkçe/turkish', 
       'tükçe',
       'türkçe (turkish)',
       'türkçe ', 'turkish (native speaker)', 'turk',
       'türkce',
       'turkish,',
       '■ türkçe ■', 'turksih', 'turkish(native)',
       'turkish - native',
       'turkish (mother tongue)']

In [ ]:
languages['language'] = languages['language'].replace(turkish, 'turkish')

<ipython-input-104-06bb064a7ab9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  languages['language'] = languages['language'].replace(turkish, 'turkish')


In [ ]:
values_to_remove = ['powerbuilder', 'vb.net', 'pl/sql','yökdi̇l 2017   85','ms sql (sorgu dili)','python',
                    'c#','java', 'coldfusion', 'rscript', 'kotlin','circassian', 'objective-c',
                     'c++', 'עברית', 'javascript', 'php', 'html','c', 'oracle data integrator',
       'hebrew', 'sql', 'assembly', 'sarcasm']

languages = languages[~languages['language'].isin(values_to_remove)]

In [ ]:
languages.loc[~languages['language'].isin(['english', 'turkish']), 'language'] = 'other'

In [ ]:
languages.language.unique()

array(['english', 'turkish', 'other'], dtype=object)

In [ ]:
languages.head()

,user_id,language,proficiency
3,10,english,NaN
4,11,turkish,3.0
5,11,english,3.0
6,11,other,3.0
7,12,turkish,3.0


#### language column

In [ ]:
# Apply one-hot encoding to the language column
one_hot = pd.get_dummies(languages['language'])

# Group by the id column and sum the one-hot encoded language columns
grouped = one_hot.groupby(languages['user_id']).sum()

# Join the one-hot encoded DataFrame with the original id column
result = languages[['user_id']].drop_duplicates().merge(grouped, on='user_id')

print(result)


       user_id  english  other  turkish
0           10        1      0        0
1           11        1      1        1
2           12        1      0        1
3           13        1      1        0
4           14        1      0        1
...        ...      ...    ...      ...
33266    66265        1      0        0
33267    66269        1      0        0
33268    66271        1      0        0
33269    66272        1      0        0
33270    66273        1      1        1

[33271 rows x 4 columns]


In [ ]:
## add cols on train and test
train_csv = pd.merge(train_csv, result, on='user_id')

test_csv = pd.merge(test_csv, result, on='user_id')


,user_id,industry,num_of_skills,num_of_experience,is_from_turkey,english,other,turkish
0,33967,Computer Software,31,2,1,1,2,1
1,55082,Internet,37,2,1,1,0,1
2,37165,Electrical/Electronic Manufacturing,4,1,1,1,0,1
3,42841,Utilities,25,2,1,1,1,1
4,17053,Computer Software,47,6,1,1,0,1


#### proficiency column EKLENMEDİ

In [ ]:
counts = languages['proficiency'].value_counts()

In [ ]:
counts

3.0    43105
1.0     7923
2.0     6530
Name: proficiency, dtype: int64

In [ ]:
# Group the DataFrame by 'id' and calculate the mean of 'value', filling nulls with the mean of the column
result_prof = languages.fillna(languages.mean()).groupby('user_id')['proficiency'].mean()

print(result_prof)

user_id
10       2.611244
11       3.000000
12       3.000000
13       2.500000
14       3.000000
           ...   
66265    2.611244
66269    2.611244
66271    2.611244
66272    3.000000
66273    2.333333
Name: proficiency, Length: 33271, dtype: float64


<ipython-input-122-c47def263e51>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  result_prof = languages.fillna(languages.mean()).groupby('user_id')['proficiency'].mean()


# Correlation matrix

In [ ]:
train_csv.head()

,user_id,industry,moved_after_2019,num_of_skills,num_of_experience,is_from_turkey,english,other,turkish,num_of_school
0,1301,6,1,23,4,1.0,1,0,1,2
1,6950,3,0,49,6,1.0,1,2,0,1
2,26046,1,0,30,4,1.0,1,0,0,3
3,11005,5,0,48,3,1.0,1,0,1,2
4,13722,2,1,12,1,1.0,1,0,0,3


#Look before modeling

In [ ]:
# Compute the correlation matrix
corr_matrix = train_csv.corr()

# Print the correlation matrix
print(corr_matrix)

                    user_id  industry  moved_after_2019  num_of_skills  \
user_id            1.000000 -0.012615         -0.023376      -0.004728   
industry          -0.012615  1.000000          0.008532       0.001879   
moved_after_2019  -0.023376  0.008532          1.000000      -0.000696   
num_of_skills     -0.004728  0.001879         -0.000696       1.000000   
num_of_experience  0.012181  0.000626          0.140682       0.269422   
is_from_turkey     0.012247 -0.003282         -0.011453       0.010376   
english            0.008151 -0.005445          0.001823       0.016110   
other              0.012815  0.011806          0.045135       0.068008   
turkish           -0.009444 -0.013672          0.041048       0.028026   
num_of_school      0.012090  0.005721         -0.016663       0.176187   

                   num_of_experience  is_from_turkey   english     other  \
user_id                     0.012181        0.012247  0.008151  0.012815   
industry                    0.000

In [ ]:
counts = train_csv['moved_after_2019'].value_counts()
print(counts)

0    14777
1    12260
Name: moved_after_2019, dtype: int64


# Model 1

In [ ]:
train_csv.isna().sum()

user_id              0
industry             6
moved_after_2019     0
num_of_skills        0
num_of_experience    0
is_from_turkey       0
english              0
other                0
turkish              0
num_of_school        0
dtype: int64

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X = train_csv.drop("moved_after_2019",axis=1)
y = train_csv["moved_after_2019"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
from xgboost import XGBClassifier

# Create the XGBoost model
xgb_model = XGBClassifier()
# Use cross-validation to evaluate the performance of the model
scores = cross_val_score(xgb_model, X_train, y_train, cv=5)
print("Cross-validation scores:", scores)

Cross-validation scores: [0.58335535 0.58599736 0.58361955 0.59418758 0.59550859]


# Model 2

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Load data
X = train_csv.drop("moved_after_2019", axis=1)
y = train_csv["moved_after_2019"]

# Split data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)

# Set parameters
params = {"objective": "binary:logistic", "max_depth": 3}

# Cross validation
cv_results = xgb.cv(
    params=params,
    dtrain=dtrain,
    num_boost_round=10,
    nfold=5,
    metrics="auc",
    as_pandas=True
)

# Print results
print(cv_results)


   train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0        0.588415       0.002003       0.582753      0.004465
1        0.597090       0.002413       0.588794      0.005925
2        0.601405       0.002122       0.594535      0.007478
3        0.604796       0.002466       0.597015      0.007057
4        0.607525       0.001920       0.599447      0.006944
5        0.610596       0.001786       0.601980      0.007195
6        0.613464       0.002080       0.602733      0.007830
7        0.615228       0.002327       0.603444      0.006865
8        0.617304       0.002432       0.604249      0.007452
9        0.619564       0.001932       0.605313      0.007839


# Model 3 RandomForest

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

# Create a preprocessing and modeling pipeline
# putting preproccessing steps together on modeling step, we use pipeline for multiple steps (combine them)
model_0 = Pipeline(steps=[("model", RandomForestClassifier())])

# Split data
X = train_csv.drop("moved_after_2019",axis=1)
y = train_csv["moved_after_2019"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Fit and score the model
model_0.fit(X_train, y_train)
model_0.score(X_val,y_val)

0.6311020710059172

#Model 4

In [ ]:
# Import required libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


# Separate target variable from features
X = train_csv.drop('moved_after_2019', axis=1)
y = train_csv['moved_after_2019']

# Define hyperparameters to tune
params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Create Random Forest model
rf_model = RandomForestClassifier()

# Tune hyperparameters using GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=params, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Print best parameters
print("Best parameters:", grid_search.best_params_)

# Create new Random Forest model with best hyperparameters
best_model = RandomForestClassifier(**grid_search.best_params_)

# Fit the model
best_model.fit(X, y)



Best parameters: {'max_depth': 7, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}


AttributeError: ignored

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

y_pred = best_model.predict(X_val)

# Calculate accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print accuracy score
print("Accuracy:", accuracy)

Accuracy: 0.6133505917159763


In [ ]:
y_pred = best_model.predict(test_csv)

In [ ]:
y_pred

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
test_csv['preds'] = y_pred 

In [ ]:
test_csv = test_csv.reindex(columns=['user_id','industry','num_of_skills','num_of_experience','is_from_turkey','english','other','turkish','num_of_school'])

In [ ]:
test_csv.head()

,user_id,industry,num_of_skills,num_of_experience,is_from_turkey,english,other,turkish,num_of_school,preds
0,33967,7,31,2,1,1,2,1,4,0
1,55082,3,37,2,1,1,0,1,2,0
2,37165,1,4,1,1,1,0,1,2,0
3,42841,1,25,2,1,1,1,1,1,0
4,17053,7,47,6,1,1,0,1,4,1


In [ ]:
test_csv.count()

user_id              6233
industry             6233
num_of_skills        6233
num_of_experience    6233
is_from_turkey       6233
english              6233
other                6233
turkish              6233
num_of_school        6233
preds                6233
dtype: int64